In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from urllib.parse import urljoin
import time

In [2]:
base_url = "https://ahli.com"
blog_url = f"{base_url}/blog"
headers = {"User-Agent": "Mozilla/5.0"}


In [3]:
folder_name = "Ahli_Blog_Images"
os.makedirs(folder_name, exist_ok=True)

response = requests.get(blog_url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")


In [4]:
category_links = []
for a in soup.select("div.wp-block-button a[href*='/category/']"):
    full_link = urljoin(base_url, a["href"])
    if full_link not in category_links:
        category_links.append(full_link)

print(f"📁 تم العثور على {len(category_links)} Category")


📁 تم العثور على 10 Category


In [9]:
all_articles = []
Bank_Name = "Ahli Bank"
for category_url in category_links:
    print(f"\n📂 Browsing the category: {category_url}")
    next_page = category_url

    while next_page:
        try:
            cat_response = requests.get(next_page, headers=headers)
            cat_soup = BeautifulSoup(cat_response.text, "html.parser")

            article_cards = cat_soup.select("li.wp-block-post")

            for card in article_cards:
                try:
                    link_tag = card.select_one("a[href*='/']")
                    if not link_tag:
                        continue
                    article_url = urljoin(base_url, link_tag["href"])

                    title_tag = card.select_one("h2")
                    title = title_tag.get_text(strip=True) if title_tag else "No Title"

                    img_tag = card.select_one("img")
                    image_url = img_tag["src"] if img_tag else ""
                    safe_title = "".join(c for c in title if c.isalnum() or c in (" ", "_")).strip().replace(" ", "_")
                    image_name = f"{safe_title[:50]}.jpg"
                    image_path = os.path.join(folder_name, image_name)

                    article_resp = requests.get(article_url, headers=headers)
                    article_soup = BeautifulSoup(article_resp.text, "html.parser")
                    paragraphs = article_soup.find_all("p")
                    content = "\n".join(p.get_text(strip=True) for p in paragraphs)

                    if image_url:
                        img_resp = requests.get(image_url)
                        with open(image_path, "wb") as f:
                            f.write(img_resp.content)

                    all_articles.append({
                        "Bank Name": Bank_Name,
                        "Category": category_url.split("/")[-2].capitalize(),
                        "Title": title,
                        "Article_Link": article_url,
                        "Content": content,
                        "Image_Name": image_name
                    })

                    print(f"✔️ {title}")

                except Exception as e:
                    print(f"❌ خطأ في مقال: {e}")
                time.sleep(0.5)

            # البحث عن رابط الصفحة التالية
            next_button = cat_soup.select_one("a.wp-block-query-pagination-next")
            if next_button and next_button.get("href"):
                next_page = urljoin(base_url, next_button["href"])
                print(f"➡️ الذهاب للصفحة التالية: {next_page}")
            else:
                next_page = None  # مافي صفحات زيادة

        except Exception as e:
            print(f"❌category خطأ في تصفح : {e}")
            next_page = None



📂 Browsing the category: https://ahli.com/category/blog/
✔️ AHLI FINTECH Partners with CloudVests to Equip FinTech Startups with Cloud Optimization Services
✔️ 5 Differences Between Debit and Credit Cards
✔️ World No Tobacco Day!
✔️ Cash vs. Easy Payment Plan, Who’s the winner?!
✔️ Earth Day ????
✔️ 5 Ways to Save Money and Stay Happy
✔️ 7 Secrets to Saving Money While Shopping
✔️ Start Early, Save Big: Why Kids Should Learn About Money Now?!
✔️ Gold … Your timeless investment!
➡️ الذهاب للصفحة التالية: https://ahli.com/category/blog/page/2/
✔️ Ahli & Wings of hope society Provide Support for Solar Energy Project and Handicrafts Workshops in Dibbin Forest Reserve

📂 Browsing the category: https://ahli.com/category/csr/
✔️ Ahli & Wings of hope society Provide Support for Solar Energy Project and Handicrafts Workshops in Dibbin Forest Reserve
✔️ The Jordan Ahli Bank organizes the “Winter Campaign” event in collaboration with the Charitable Clothing Bank
✔️ The joining of Jordan Ahli Ban

In [10]:
df = pd.DataFrame(all_articles)
df

,Bank Name,Category,Title,Article_Link,Content,Image_Name
0,Ahli Bank,Blog,AHLI FINTECH Partners with CloudVests to Equip...,https://ahli.com/ahli-fintech-partners-with-cl...,"Amman, August 2024:AHLI FINTECH– a fully owned...",AHLI_FINTECH_Partners_with_CloudVests_to_Equip...
1,Ahli Bank,Blog,5 Differences Between Debit and Credit Cards,https://ahli.com/5-differences-between-debit-a...,Have you ever thought about the difference bet...,5_Differences_Between_Debit_and_Credit_Cards.jpg
2,Ahli Bank,Blog,World No Tobacco Day!,https://ahli.com/world-no-tobacco-day/,"\n\nEvery year on May 31, people around the wo...",World_No_Tobacco_Day.jpg
3,Ahli Bank,Blog,"Cash vs. Easy Payment Plan, Who’s the winner?!",https://ahli.com/cash-vs-easy-payment-plan/,"When you’re out shopping, do you ever wonder, ...",Cash_vs_Easy_Payment_Plan_Whos_the_winner.jpg
4,Ahli Bank,Blog,Earth Day ????,https://ahli.com/earth-day-%f0%9f%8c%8d/,"Hey there! Today, let’s dive into Earth Day an...",Earth_Day.jpg
...,...,...,...,...,...,...
111,Ahli Bank,Uncategorized,Ahli salary account winners,https://ahli.com/ahli-salary-account-winners/,Congratulations to the winners of our Ahli sal...,Ahli_salary_account_winners.jpg
112,Ahli Bank,Uncategorized,Relocation of Ahli Brokerage,https://ahli.com/ahli-brokerage-new-location/,تعلن شركة الأهلي للوساطة المالية عن انتقالها ل...,Relocation_of_Ahli_Brokerage.jpg
113,Ahli Bank,Uncategorized,AHLI FINTECH leads seed investment with ISSF i...,https://ahli.com/ahli-fintech-leads-seed-inves...,AHLI FINTECH– the financial technology and inn...,AHLI_FINTECH_leads_seed_investment_with_ISSF_i...
114,Ahli Bank,Uncategorized,Credit Rating; what makes a credit rating good...,https://ahli.com/credit-rating1/,Credit Rating:An indicator that financial comp...,Credit_Rating_what_makes_a_credit_rating_good_...
